In [1]:
!pip install langsmith

In [7]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["COHERE_API_KEY"] = "2zPsEcvQ1tzELqmw0ORVcVEX51H5p7KXLrpaqmrz"

In [4]:
from langsmith import Client

# QA
inputs = [
    "What are the names of the passengers?",
    "What is the airline's name and flight number?",
    "Can you provide me the flight routes along with the time?",
    "Can you provide me with the ticket number of the passengers?",
    "What are the flight duration for all the flights mentioned?",
    "What are the baggage limits?",
    "What are the date of birth of the passengers?",
    "What are the ticket numbers?"
]

outputs = [
    "",
    "",
    "",
    "",
    "",
    "",
    "",
    "",
]

qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# Create dataset
client = Client()
dataset_name = "ticket_test_evaluation_hallucination"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about ticket.",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [8]:
from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

In [14]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

answer_hallucination_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": """Is the Assistant's Answer grounded in the Ground Truth documentation? A score of [[1]] means that the
            Assistant answer contains is not at all based upon / grounded in the Groun Truth documentation. A score of [[5]] means 
            that the Assistant answer contains some information (e.g., a hallucination) that is not captured in the Ground Truth 
            documentation. A score of [[10]] means that the Assistant answer is fully based upon the in the Ground Truth documentation."""
        },
        # If you want the score to be saved on a scale from 0 to 1
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["contexts"],
        "input": example.inputs["question"],
    },
)

WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.


ValueError: Evaluation with the <class 'langchain.evaluation.scoring.eval_chain.LabeledScoreStringEvalChain'> requires a language model to function. Failed to create the default 'gpt-4' model. Please manually provide an evaluation LLM or check your openai credentials.

In [15]:
dataset_name = "ticket_test_evaluation_hallucination"
experiment_results = evaluate(
    {"answer": model,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="rag-qa-cohere-hallucination",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "ticket_test_evaluation_hallucination_cohere",
    },
)

NameError: name 'predict_rag_answer_with_context' is not defined